In [2]:
import xgboost as xgb

import pandas as pd
import pickle
import numpy as np
import math
import pymongo

%pylab inline
pd.set_option('display.mpl_style', 'default')
figsize(12, 9)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from pymongo import MongoClient, DESCENDING

# from pymongo.objectid import ObjectId
import bson

import os
import json

from bson.objectid import ObjectId

Populating the interactive namespace from numpy and matplotlib


In [11]:
client = MongoClient()
db = client.dataframe

db_users_and_features = client['users_and_features']
posts = db_users_and_features.posts

In [15]:
new_embds = []
for e in posts.find({'profile_embedding': { '$exists': True}}):
    new_embds.append(e['profile_embedding'])

In [14]:
# posts.find_one({'profile_embedding': { '$exists': True}})

In [16]:
new_embds = np.array(new_embds)

In [21]:
new_embds.shape

(117272, 307)

In [18]:
from sklearn.decomposition import PCA

In [19]:
pca_model = PCA(n_components=5)

In [20]:
pca_model.fit(new_embds)

PCA(copy=True, iterated_power='auto', n_components=5, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [47]:
import cPickle

In [48]:
# save the classifier
with open('pca_model.pkl', 'wb') as fid:
    cPickle.dump(pca_model, fid)    

In [51]:
for e in posts.find({'profile_embedding': {'$exists': True},
                     'face_features':  {'$exists': True},
                     'interests_vector': {'$exists': False}}):    
#     print json.dumps(list(pca_model.transform(e['profile_embedding'])[0]))
#     break
    posts.update(
        { u'_id': e[u'_id'] },
        { '$set': { 'interests_vector': json.dumps(list(pca_model.transform(e['profile_embedding'])[0])) } }
    )